In [1]:
!git clone https://github.com/KuzmaKhrabrov/character-tokenizer.git

Cloning into 'character-tokenizer'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 20 (delta 5), reused 10 (delta 3), pack-reused 0
Receiving objects: 100% (20/20), 5.89 KiB | 1.96 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
!pip install transformers

In [3]:
import string
import sys
sys.path.append("character-tokenizer")
from charactertokenizer import CharacterTokenizer

chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"
model_max_length = 64
tokenizer = CharacterTokenizer(chars, model_max_length)

In [4]:
len(chars)

66

In [5]:
example = "Привет"
tokens = tokenizer(example)
print(tokens)

{'input_ids': [0, 39, 42, 26, 12, 18, 46, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [6]:
example = "Привет"
tokens = tokenizer.encode_plus(
    example,
    add_special_tokens = True,
    max_length = 20,
    padding = 'max_length'
)
print(tokens)

{'input_ids': [0, 39, 42, 26, 12, 18, 46, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [ ]:
#!rm -rf all_accents.zip

In [7]:
!wget https://github.com/Koziev/NLP_Datasets/raw/master/Stress/all_accents.zip
!unzip all_accents.zip

--2023-11-23 07:33:29--  https://github.com/Koziev/NLP_Datasets/raw/master/Stress/all_accents.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Koziev/NLP_Datasets/master/Stress/all_accents.zip [following]
--2023-11-23 07:33:29--  https://raw.githubusercontent.com/Koziev/NLP_Datasets/master/Stress/all_accents.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10624775 (10M) [application/zip]
Saving to: ‘all_accents.zip’

all_accents.zip     100%[===================>]  10.13M  --.-KB/s    in 0.05s   

2023-11-23 07:33:30 (193 MB/s) - ‘all_accents.zip’ saved [10624775/10624775]

Archive:  all_a

In [8]:
import pandas as pd
file_path = 'all_accents.tsv'
df = pd.read_csv(file_path, delimiter='\t')

df.sample(20)

,-де,-д^е
881120,осеняющего,осен^яющего
775361,непридыхательное,непридых^ательное
368921,жалобных,ж^алобных
1128677,пошатываниях,пош^атываниях
1525112,увильнувшем,увильн^увшем
1010138,питбультэрьер,питбультэрь^ер
1474477,тактильных,такт^ильных
1494800,томашевич,том^ашевич
108606,боронится,борон^ится
951043,пассажирском,пассаж^ирском


In [9]:
len(df)

1680534

In [10]:
max_length = df['-де'].apply(len).max()

longest_string = df[df['-де'].apply(len) == max_length]['-де'].values[0]

print("Самая длинная строка:", longest_string)
print("Длина строки:", max_length)

Самая длинная строка: лланвайрпуллгуингиллгогерихуирндробуллллантисилиогогогох
Длина строки: 56


In [11]:
df.iloc[624999]

-де     лланвайрпуллгуингиллгогерихуирндробуллллантиси...
-д^е    лланвайрпуллгуингиллгогерихуирндробуллллантиси...
Name: 624999, dtype: object

*Жесткач :)*

In [12]:
import torch
from torch.utils.data import Dataset

In [13]:
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
class StressDataset(Dataset):
    def __init__(self, file_path, max_length=56, test_size=0.5, random_seed=42):
        self.df = pd.read_csv(file_path, delimiter='\t')
        self.chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"
        self.model_max_length = 64
        self.tokenizer = CharacterTokenizer(self.chars, self.model_max_length)
        self.word_max_length = max_length

        self.train_data_index, self.test_data_index = (
            train_test_split(
                self.df.index,
                test_size=test_size,
                random_state=random_seed
            )
        )
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        text = self.df.loc[idx, '-де']
        label = self.df.loc[idx, '-д^е']

        tokens = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.word_max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        # [CLS]востфак[SEP][PAD]...[PAD]
        # востфак востф^ак
        number_stress_token = label.find('^') + 1
        labels = [0]*len(tokens['attention_mask'].squeeze())
        labels[number_stress_token] = 1
        
        labels = torch.tensor(labels, dtype=torch.long)
        tokens['labels'] = labels
        return tokens
        

In [15]:
file_path = 'all_accents.tsv'
stress_dataset = StressDataset(file_path)

In [16]:
INDEX = 107500
word_tokens = stress_dataset[INDEX]
(
word_tokens['labels'],
df.iloc[INDEX].values[0],
df.iloc[INDEX].values[1],
tokenizer.decode(word_tokens['input_ids'].squeeze().tolist())
)

(tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'бордель',
 'борд^ель',
 '[CLS]бордель[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]')

In [17]:
from torch.utils.data import Subset, DataLoader

In [18]:
file_path = 'all_accents.tsv'
stress_dataset = StressDataset(file_path)

In [19]:
def collate_fn(batch):
    input_ids = torch.stack([sample['input_ids'] for sample in batch])
    attention_mask = torch.stack([sample['attention_mask'] for sample in batch])
    labels = torch.stack([sample['labels'] for sample in batch])

    return [input_ids.squeeze(1), attention_mask.squeeze(1), labels]

batch_size = 256

train_loader = DataLoader(
    Subset(stress_dataset, stress_dataset.train_data_index),
    batch_size=batch_size,
    shuffle=True,
    drop_last = True,
    collate_fn = collate_fn,
)

test_loader = DataLoader(
    Subset(stress_dataset, stress_dataset.test_data_index),
    batch_size=batch_size,
    shuffle=False,
    collate_fn = collate_fn,
)

In [51]:
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [21]:
!pip install transformers
!pip install pynvml

In [22]:
#TEST
cnt = 0
for batch in train_loader:
    print(len(batch[0]), len(batch[1]), len(batch[2]))
    print(type(batch[0]), type(batch[1]), type(batch[2]))
    cnt += 1
    if cnt == 3:
        break

256 256 256
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
256 256 256
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
256 256 256
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>


In [23]:
#TEST
for batch in train_loader:
    print(batch[0].shape, batch[1].shape, batch[2].shape)
    break

torch.Size([256, 56]) torch.Size([256, 56]) torch.Size([256, 56])


In [24]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [25]:
from transformers import get_linear_schedule_with_warmup

In [26]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForTokenClassification, AdamW

from transformers import BertConfig

config = BertConfig(
    hidden_size=512,
    num_labels=2,  # 2 класса: "NO", "PRIMARY"
    output_attentions=False,
    output_hidden_states=False,
    num_attention_heads=8,
    num_hidden_layers=12,
    max_position_embeddings=60,
    pad_token_id=0,
    id2label={
        "0": "NO",
        "1": "PRIMARY",
    },
    label2id={
        "NO": 0,
        "PRIMARY": 1,
    },
)

model = BertForTokenClassification(config)
model.cuda()


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(60, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, ele

In [27]:
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)
epochs = 5

total_steps = len(train_loader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
criterion = torch.nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [28]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels, mask):
    slices = []
    for m in mask:
        tmp = np.where(np.flip(m) == 1)[0][0]
        index_one_from_end = len(m) - 1 - tmp
        slices.append((1, index_one_from_end))
        
    pred_labels = np.argmax(preds, axis=-1)
    
    t_pn = 0
    for i in range(len(labels)):
        start = slices[i][0]
        end = slices[i][1]
        len_ = end - start
        t_pn += (np.sum(pred_labels[i, start:end] == labels[i, start:end]) == len_)

    return t_pn / labels.shape[0]


In [46]:
X = np.array([
     [[0.3, 0.7],[0.3, 0.7], [0.8, 0.2], [0.8, 0.2],[1.0, 0.7], [0.1, 0.9]],
     [[0.3, 0.7],[0.6, 0.4], [0.2, 0.8], [0.8, 0.2],[0.3, 0.7], [0.1, 0.9]],
     [[0.3, 0.7],[0.3, 0.7], [0.8, 0.2], [0.8, 0.2],[0.3, 0.7], [0.1, 0.9]],
    ])
MASK = np.array([[1,1,1,1,0,0],
                 [1,1,1,1,1,0],
                 [1,1,1,1,1,0]])

Y = np.array([[0,1,0,0,0,0],
              [0,0,1,0,0,0],
              [0,1,0,0,0,0]])

flat_accuracy(X,Y,MASK)

1.0

In [ ]:
# TEST
#for batch in train_loader:
#    with torch.no_grad():
#        output = model(
#            batch[0].to(device),
#            token_type_ids=None,
#            attention_mask=batch[1].to(device),
#            labels=batch[2].to(device)
#        )
#    
#    logits = output.logits.detach().cpu().numpy()
#    label_ids = batch[2].to('cpu').numpy()
#    break
#

In [ ]:
#logits.shape

In [ ]:
#label_ids.shape

In [ ]:
#np.argmax(logits, axis=-1)[0], label_ids[0]


In [45]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

## пайплайн обучения

In [ ]:
import random
import numpy as np

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


training_stats = []


total_t0 = time.time()

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()

    total_train_loss = 0
    total_train_accuracy = 0
    
    model.train()

    for step, batch in enumerate(train_loader):
        # Progress update every 100 batches.
        if step % 500 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_loader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
        output = model(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask,
            labels=b_labels,
            return_dict=True
        )

        total_train_loss += output.loss.item()

        output.loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        logits = output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        b_mask = b_input_mask.to('cpu').numpy()

        batch_acc = flat_accuracy(logits, label_ids, b_mask)
        if step % 500 == 0 and not step == 0:
            print(f'  Batch accuracy = {batch_acc} ')
        total_train_accuracy += batch_acc


    avg_train_accuracy = total_train_accuracy / len(train_loader)

    avg_train_loss = total_train_loss / len(train_loader)

    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Accuracy: {0:.2f}".format(avg_train_accuracy))
    print("  Training epoch took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()


    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in test_loader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():


            output = model(b_input_ids,
                           token_type_ids=None,
                           attention_mask=b_input_mask,
                           labels=b_labels)

        # Accumulate the validation loss.
        total_eval_loss += output.loss.item()

        # Move logits and labels to CPU
        logits = output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        b_mask = b_input_mask.to('cpu').numpy()
        

        total_eval_accuracy += flat_accuracy(logits, label_ids, b_mask)


    avg_val_accuracy = total_eval_accuracy / len(test_loader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(test_loader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

# Accuracy: 0.91

* **сохранил веса с прошлого запуска (учил 5 эпох)***

In [40]:
model_save_path = "./bert_model"

In [ ]:
model.save_pretrained(model_save_path)

In [ ]:
#!zip -r bert_model.zip ./bert_model

In [37]:
#!rm -rf weights.zip

In [38]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1hrLQGpZ_DD1j9n-q7fpAWMEe0mDan7tD' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1hrLQGpZ_DD1j9n-q7fpAWMEe0mDan7tD" -O weights.zip && rm -rf /tmp/cookies.txt
!unzip weights.zip

--2023-11-23 08:02:03--  https://docs.google.com/uc?export=download&confirm=t&id=1hrLQGpZ_DD1j9n-q7fpAWMEe0mDan7tD
Resolving docs.google.com (docs.google.com)... 173.194.69.139, 173.194.69.138, 173.194.69.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.69.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fj60tmk38cl6sqhl9stsdilg17emf2k1/1700726475000/09074642392443333439/*/1hrLQGpZ_DD1j9n-q7fpAWMEe0mDan7tD?e=download&uuid=09dc664d-af18-4528-98bf-7b1d1441df9c [following]
--2023-11-23 08:02:03--  https://doc-0o-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fj60tmk38cl6sqhl9stsdilg17emf2k1/1700726475000/09074642392443333439/*/1hrLQGpZ_DD1j9n-q7fpAWMEe0mDan7tD?e=download&uuid=09dc664d-af18-4528-98bf-7b1d1441df9c
Resolving doc-0o-38-docs.googleusercontent.com (doc-0o-38-docs.googleusercontent.com)... 108.177.127.132,

In [53]:
loaded_model = BertForTokenClassification.from_pretrained(model_save_path)
loaded_model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(60, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, ele

In [54]:
def model_eval(model):
    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in test_loader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():


            output = model(b_input_ids,
                           token_type_ids=None,
                           attention_mask=b_input_mask,
                           labels=b_labels)

        # Accumulate the validation loss.
        total_eval_loss += output.loss.item()

        # Move logits and labels to CPU
        logits = output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        b_mask = b_input_mask.to('cpu').numpy()
        

        total_eval_accuracy += flat_accuracy(logits, label_ids, b_mask)


    avg_val_accuracy = total_eval_accuracy / len(test_loader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(test_loader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

In [55]:
model_eval(loaded_model)


Running Validation...
  Accuracy: 0.91
  Validation Loss: 0.01
  Validation took: 0:17:29


*Вот доказательство*

In [152]:
def test_suggestion(model, suggestion: str) -> str:
    word_max_length=56
    words = suggestion.split()
    model_max_length = 64
    chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"
    tokenizer = CharacterTokenizer(chars, model_max_length)
    input_ids = []
    input_mask = []
    for word in words:
        tokens = tokenizer.encode_plus(
            word,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids.append(tokens['input_ids'].squeeze(1))
        input_mask.append(tokens['attention_mask'].squeeze(1))
    
    input_ids = torch.stack(input_ids, dim=1).squeeze(0).to(device)
    input_mask = torch.stack(input_mask, dim=1).squeeze(0).to(device)

    with torch.no_grad():
        output = model(
            input_ids,
            token_type_ids=None,
            attention_mask=input_mask,
        )
    logits = output.logits.detach().cpu().numpy()
    pred_labels = np.argmax(logits, axis=-1)
    
    slices = []
    for in_m in input_mask:
        m = np.array(in_m.cpu())
        tmp = np.where(np.flip(m) == 1)[0][0]
        index_one_from_end = len(m) - 1 - tmp
        slices.append((1, index_one_from_end))
    
    for i in range(len(words)):
        start = slices[i][0]
        end = slices[i][1]
        predicat = pred_labels[i][start:end]
        try:
            stress_index = np.where(np.array(predicat) == 1)[0][0]
        except:
            continue
        
        words[i] = words[i][:stress_index] + words[i][stress_index].upper() + words[i][stress_index+1:]
    
    return " ".join(words)
        
        
    

In [154]:
test_suggestion(loaded_model, 'чтобы торты были вкусные нужно добавить сливовый творог')

'чтОбы тОрты бЫли вкуснЫе нУжно добАвить слИвовый твОрог'

In [155]:
test_suggestion(loaded_model, 'как ставить ударения в разных частях речи')

'кАк стАвить ударЕния в рАзных чАстях рЕчи'